In [1]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Nombre archivos

In [3]:
dir_archivo = '/home/tonatiuh/Documents/Desarrollo/ZophiaLearning/ejercicios/'
dir_complemento = 'prestamos/stage/'

In [4]:
nombre_archivos = [
    'prestamos_bancos.parquet',
    'prestamos_destinatarios.parquet',
    'prestamos_pagos.parquet',
    'prestamos_plaza.parquet',
    'prestamos_solicitudes.parquet',
    'prestamos_solicitudes_procesadas.parquet']

### Funciones

In [5]:
def df_almacenamiento_parquet(dir_archivo, nombre_archivo, df):
    nombre_destino = f'prestamos/curated/{nombre_archivo}'
    df.write.mode('overwrite').parquet(dir_archivo+nombre_destino)
    print(nombre_destino)

    
def df_almacenamiento_csv(nombre_archivo, df):
    df_filtrado = df.limit(10)
    df_pandas = df_filtrado.toPandas()
    nombre_output = nombre_archivo.replace('.parquet', '')
    nombre_csv = f'output/{nombre_output}.csv'
    df_pandas.to_csv(nombre_csv, index=False)
    print(nombre_csv)


def df_almacenamiento(dir_archivo, nombre_archivo, df):
    df_almacenamiento_parquet(dir_archivo, nombre_archivo, df)
    df_almacenamiento_csv(nombre_archivo, df)

    
letras_acento = 'áéíóúÁÉÍÓÚ'
letras_sin_acento = 'aeiouAEIOU'
tabla_acentos = str.maketrans(letras_acento, letras_sin_acento)

### Tabla prestamos_bancos

In [6]:
nombre_archivo = nombre_archivos[0]
df_bancos = spark.read.format('parquet')\
                .load(dir_archivo+dir_complemento+nombre_archivo)

for col_nombre in ['digitos', 'BANCO']:
    df_bancos = df_bancos.withColumn(col_nombre, F.translate(col_nombre, "'", ""))
    df_bancos = df_bancos.withColumnRenamed(col_nombre, col_nombre.upper())

In [7]:
df_bancos.printSchema()

root
 |-- DIGITOS: string (nullable = true)
 |-- BANCO: string (nullable = true)



In [8]:
df_bancos.show(5)

+-------+-------------+
|DIGITOS|        BANCO|
+-------+-------------+
|    002|      BANAMEX|
|    006|    BANCOMEXT|
|    009|     BANOBRAS|
|    012|BBVA BANCOMER|
|    014|    SANTANDER|
+-------+-------------+
only showing top 5 rows



In [9]:
df_almacenamiento(dir_archivo, nombre_archivo, df_bancos)

prestamos/curated/prestamos_bancos.parquet
output/prestamos_bancos.csv


### Tabla prestamos_destinatarios

In [10]:
nombre_archivo = nombre_archivos[1]
df_destinatarios = spark.read.format('parquet')\
                        .load(dir_archivo+dir_complemento+nombre_archivo)


for col_nombre in ['ID', 'banco', 'plaza']:
    df_destinatarios = df_destinatarios.withColumn(col_nombre,
                                                   F.col(col_nombre).cast(IntegerType()))
    
    df_destinatarios = df_destinatarios.withColumnRenamed(col_nombre,
                                                          col_nombre.upper())

    
for col_nombre in ['cp', 'fecha_nacimiento', 'correo', 'clabe']:
    df_destinatarios = df_destinatarios.withColumnRenamed(col_nombre,
                                                          col_nombre.upper())


for col_nombre in ['nombre', 'direccion', 'empleo', 'empleador']:
    df_destinatarios = df_destinatarios.withColumn(col_nombre, 
                                                   F.translate(col_nombre,
                                                               letras_acento,
                                                               letras_sin_acento))
    
    df_destinatarios = df_destinatarios.withColumn(col_nombre,
                                                   F.translate(col_nombre, '\n', ''))
    
    df_destinatarios = df_destinatarios.withColumnRenamed(col_nombre,
                                                          col_nombre.upper())

In [11]:
df_destinatarios.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- DIRECCION: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- FECHA_NACIMIENTO: string (nullable = true)
 |-- EMPLEO: string (nullable = true)
 |-- EMPLEADOR: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- BANCO: integer (nullable = true)
 |-- PLAZA: integer (nullable = true)
 |-- CLABE: long (nullable = true)



In [12]:
df_destinatarios.show(1, vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------------------
 ID               | 0                                                     
 NOMBRE           | Sr(a). Luz Leon                                       
 DIRECCION        | Diagonal Argelia 602 655San Caridad de la Montaña, BC 
 CP               | 76193                                                 
 FECHA_NACIMIENTO | 1960-12-12                                            
 EMPLEO           | Biomedical engineer                                   
 EMPLEADOR        | Najera y Estrada e Hijos                              
 CORREO           | pablo10@gmail.com                                     
 BANCO            | 147                                                   
 PLAZA            | 396                                                   
 CLABE            | 25944813986                                           
only showing top 1 row



In [13]:
df_almacenamiento(dir_archivo, nombre_archivo, df_destinatarios)

prestamos/curated/prestamos_destinatarios.parquet
output/prestamos_destinatarios.csv


### Tabla pagos

In [14]:
nombre_archivo = nombre_archivos[2]
df_pagos = spark.read.format('parquet')\
                .load(dir_archivo+dir_complemento+nombre_archivo)
df_pagos = df_pagos.select('fecha', 'cantidad', 'prestamo')


col_nombre = 'fecha'
df_pagos = df_pagos.withColumnRenamed(col_nombre,
                                      col_nombre.upper())


col_nombre = 'cantidad'
df_pagos = df_pagos.withColumn(col_nombre,
                               F.col(col_nombre).cast(DoubleType()))

df_pagos = df_pagos.withColumnRenamed(col_nombre,
                                      col_nombre.upper())


col_nombre = 'prestamo'
df_pagos = df_pagos.withColumn(col_nombre,
                               F.translate(col_nombre, '[]', ''))

df_pagos = df_pagos.withColumnRenamed(col_nombre,
                                      col_nombre.upper())

In [15]:
df_pagos.printSchema()

root
 |-- FECHA: string (nullable = true)
 |-- CANTIDAD: double (nullable = true)
 |-- PRESTAMO: string (nullable = true)



In [16]:
df_pagos.show(5)

+----------+--------+--------+
|     FECHA|CANTIDAD|PRESTAMO|
+----------+--------+--------+
|2017-02-14| 13100.0|     381|
|2017-02-15|  2500.0|     273|
|2017-02-27|  5000.0|     381|
|2017-03-01|  2700.0|    1662|
|2017-03-13|  6000.0|    1662|
+----------+--------+--------+
only showing top 5 rows



In [17]:
df_almacenamiento(dir_archivo, nombre_archivo, df_pagos)

prestamos/curated/prestamos_pagos.parquet
output/prestamos_pagos.csv


### Tabla plaza

In [18]:
nombre_archivo = nombre_archivos[3]
df_plaza = spark.read.format('parquet')\
                .load(dir_archivo+dir_complemento+nombre_archivo)

col_nombre = 'digitos'
df_plaza = df_plaza.withColumn(col_nombre,
                               F.translate(col_nombre, "'", ""))

df_plaza = df_plaza.withColumnRenamed(col_nombre,
                                      col_nombre.upper())

col_nombre = 'PLAZA'
df_plaza = df_plaza.withColumn(col_nombre,
                               F.translate(col_nombre, "'", ""))

df_plaza = df_plaza.withColumn(col_nombre, 
                               F.translate(col_nombre,
                                           letras_acento,
                                           letras_sin_acento))

df_plaza = df_plaza.withColumnRenamed(col_nombre,
                                      col_nombre.upper())

In [19]:
df_plaza.printSchema()

root
 |-- DIGITOS: string (nullable = true)
 |-- PLAZA: string (nullable = true)



In [20]:
df_plaza.show(5)

+-------+--------------+
|DIGITOS|         PLAZA|
+-------+--------------+
|    010|Aguascalientes|
|    314|       Zimapan|
|    640|      Zimatlan|
|    012|      Calvillo|
|    320|      El Salto|
+-------+--------------+
only showing top 5 rows



In [21]:
df_almacenamiento(dir_archivo, nombre_archivo, df_plaza)

prestamos/curated/prestamos_plaza.parquet
output/prestamos_plaza.csv


### Tabla solicitudes

In [22]:
nombre_archivo = nombre_archivos[4]
df_solicitudes = spark.read.format('parquet')\
                        .load(dir_archivo+dir_complemento+nombre_archivo)


for col_nombre in ['ID', 'fecha', 'fecha_in', 'fecha_lim',
                   'monto', 'taza', 'fecha_pago']:
    df_solicitudes = df_solicitudes.withColumnRenamed(col_nombre,
                                                      col_nombre.upper())

    
col_nombre = 'descripcion'
df_solicitudes = df_solicitudes.withColumn(col_nombre,
                                           F.col(col_nombre).cast(FloatType()))

df_solicitudes = df_solicitudes.withColumnRenamed(col_nombre,
                                                  col_nombre.upper())


for col_nombre in ['producto', 'solicitante']:
    df_solicitudes = df_solicitudes.withColumn(col_nombre, 
                                               F.translate(col_nombre,
                                                           letras_acento,
                                                           letras_sin_acento))
    
    df_solicitudes = df_solicitudes.withColumnRenamed(col_nombre,
                                                      col_nombre.upper())

In [23]:
df_solicitudes.printSchema()

root
 |-- ID: long (nullable = true)
 |-- FECHA: string (nullable = true)
 |-- FECHA_IN: string (nullable = true)
 |-- FECHA_LIM: string (nullable = true)
 |-- MONTO: long (nullable = true)
 |-- PRODUCTO: string (nullable = true)
 |-- TAZA: double (nullable = true)
 |-- DESCRIPCION: float (nullable = true)
 |-- FECHA_PAGO: string (nullable = true)
 |-- SOLICITANTE: string (nullable = true)



In [24]:
df_solicitudes.show(2, vertical=True, truncate=False)

-RECORD 0--------------------------
 ID          | 0                   
 FECHA       | 2017-06-14          
 FECHA_IN    | 2017-06-14          
 FECHA_LIM   | 2018-07-09          
 MONTO       | 255000              
 PRODUCTO    | monina              
 TAZA        | 4.5                 
 DESCRIPCION | null                
 FECHA_PAGO  | 2018-07-09          
 SOLICITANTE | Oliver Correa Tello 
-RECORD 1--------------------------
 ID          | 1                   
 FECHA       | 2017-03-21          
 FECHA_IN    | 2017-03-21          
 FECHA_LIM   | 2017-08-18          
 MONTO       | 84000               
 PRODUCTO    | basico              
 TAZA        | 8.1                 
 DESCRIPCION | null                
 FECHA_PAGO  | 2017-08-18          
 SOLICITANTE | Augusto Olmos       
only showing top 2 rows



In [25]:
df_almacenamiento(dir_archivo, nombre_archivo, df_solicitudes)

prestamos/curated/prestamos_solicitudes.parquet
output/prestamos_solicitudes.csv


### Tabla solicitudes_procesadas

In [26]:
nombre_archivo = nombre_archivos[5]
df_solicitudes_p = spark.read.format('parquet')\
                        .load(dir_archivo+dir_complemento+nombre_archivo)


for col_nombre in ['fecha_aprobado', 'fecha_solicitud', 'fecha_limite',
                   'monto_aprobado', 'solicitud_id', 'respuesta']:
    df_solicitudes_p = df_solicitudes_p.withColumnRenamed(col_nombre,
                                                          col_nombre.upper())


col_nombre = 'monto_solicitado'
df_solicitudes_p = df_solicitudes_p.withColumn(col_nombre,
                                               F.col(col_nombre).cast(DoubleType()))

df_solicitudes_p = df_solicitudes_p.withColumnRenamed(col_nombre,
                                                      col_nombre.upper())

In [27]:
df_solicitudes_p.printSchema()

root
 |-- FECHA_APROBADO: string (nullable = true)
 |-- FECHA_SOLICITUD: string (nullable = true)
 |-- FECHA_LIMITE: string (nullable = true)
 |-- MONTO_SOLICITADO: double (nullable = true)
 |-- MONTO_APROBADO: double (nullable = true)
 |-- SOLICITUD_ID: long (nullable = true)
 |-- RESPUESTA: boolean (nullable = true)



In [28]:
df_solicitudes_p.show(2, vertical=True, truncate=False)

-RECORD 0----------------------
 FECHA_APROBADO   | 2017-07-18 
 FECHA_SOLICITUD  | 2017-07-02 
 FECHA_LIMITE     | 2018-07-09 
 MONTO_SOLICITADO | 255000.0   
 MONTO_APROBADO   | 0.0        
 SOLICITUD_ID     | 0          
 RESPUESTA        | false      
-RECORD 1----------------------
 FECHA_APROBADO   | 2017-04-29 
 FECHA_SOLICITUD  | 2017-04-10 
 FECHA_LIMITE     | 2017-08-18 
 MONTO_SOLICITADO | 84000.0    
 MONTO_APROBADO   | 80000.0    
 SOLICITUD_ID     | 1          
 RESPUESTA        | true       
only showing top 2 rows



In [29]:
df_almacenamiento(dir_archivo, nombre_archivo, df_solicitudes_p)

prestamos/curated/prestamos_solicitudes_procesadas.parquet
output/prestamos_solicitudes_procesadas.csv
